# Standard QFT circuit

We used this to write the circuit for the QFT of any number n of qbits

In [1]:
# There's a dependency in Bloqade that (currently)
# spits out some warnings you don't need to worry about.
import warnings
warnings.filterwarnings("ignore")

from bloqade.qasm2.rewrite.native_gates import RydbergGateSetRewriteRule
from kirin.rewrite import Walk
from bloqade import qasm2
from bloqade.qasm2.parse.lowering import QASM2
from bloqade.qasm2.passes import QASM2Py
from bloqade.qasm2.emit import QASM2 # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer

from kirin import ir
from qiskit import QuantumCircuit

from bloqade.qasm2.rewrite.native_gates import RydbergGateSetRewriteRule
from kirin import ir
from kirin.rewrite import Walk
from bloqade.qasm2.passes import UOpToParallel, QASM2Fold

from bloqade.qasm2.emit import QASM2 # the QASM2 target
from bloqade.qasm2.parse import pprint # the QASM2 pretty printer

target = QASM2(allow_parallel=True)

import numpy as np


@ir.dialect_group(qasm2.extended)
def extended_opt(self):
    native_rewrite = Walk(RydbergGateSetRewriteRule(self)) # use Kirin's functionality to walk code line by line while applying neutral-atom gate decomposition as defined in Bloqade
    parallelize_pass = UOpToParallel(self) # review the code and apply parallelization using a heuristic
    agg_fold = QASM2Fold(self) # supports parallelization by unfolding loops to search for parallelization opportunities

    # here we define our new compiler pass
    def run_pass(
        kernel: ir.Method,
        *,
        fold: bool = True,
        typeinfer: bool = True,
        parallelize: bool = False,
    ):
        assert qasm2.extended.run_pass is not None
        qasm2.extended.run_pass(kernel, fold=fold, typeinfer=typeinfer) # apply the original run_pass to the lowered kernel
        native_rewrite.rewrite(kernel.code) # decompose all gates in the circuit to neutral atom gate set

        # here goes our parallelization optimizer; the order of the commands here matters!
        if parallelize:
            agg_fold.fixpoint(kernel)
            parallelize_pass(kernel)

    return run_pass


# helper to go from Method → Qiskit
def method_to_qiskit(method: ir.Method, *,parallelize: bool = True) -> QuantumCircuit:
    # run extended pass in case forgot
    extended_opt.run_pass(method, parallelize=parallelize)
    # emit OpenQASM2 text
    qasm = QASM2().emit_str(method)
    # parse into a Qiskit circuit
    return QuantumCircuit.from_qasm_str(qasm)

def method_to_qiskit_noparallel(method: ir.Method, *,parallelize: bool = True) -> QuantumCircuit:
    # emit OpenQASM2 text
    qasm = QASM2().emit_str(method)
    # parse into a Qiskit circuit
    return QuantumCircuit.from_qasm_str(qasm)

In [2]:
def method(n: int, parallelize: bool = False):

    @extended_opt(parallelize=parallelize)
    def method_program():
        qreg = qasm2.qreg(n)
        for m in range(n-1,-1,-1):
            qasm2.h(qreg[m])
            for k in range(n-1,-1,-1):
                if k < m:
                    qasm2.cx(qreg[k],qreg[m])
                    qasm2.rz(qreg[m],-np.pi/2**(m+1-k))
                    qasm2.cx(qreg[k],qreg[m])
                    qasm2.rz(qreg[m],np.pi/2**(m+1-k))
        return qreg

    return method_program

In [3]:
n = 12
qasm2.parse.pprint(target.emit(method(n, parallelize=False)))

KIRIN {func,lowering.call,lowering.func,py.ilist,qasm2.core,qasm2.expr,qasm2.indexing,qasm2.parallel,qasm2.uop,scf};
include "qelib1.inc";
qreg qreg[12];
U(1.5707963267949, 0.0, 3.14159265358979) qreg[11];
U(1.5707963267949, 0.0, 6.28318530717959) qreg[11];
cz qreg[10], qreg[11];
U(1.5707963267949, 3.14159265358979, 3.14159265358979) qreg[11];
U(0.0, 0.0, 3.14159265358979) qreg[10];
U(0.0, 0.0, 6.28318530717958) qreg[11];
U(0.0, 0.0, -0.7853981633974483) qreg[11];
U(1.5707963267949, 0.0, 6.28318530717959) qreg[11];
cz qreg[10], qreg[11];
U(1.5707963267949, 3.14159265358979, 3.14159265358979) qreg[11];
U(0.0, 0.0, 3.14159265358979) qreg[10];
U(0.0, 0.0, 6.28318530717958) qreg[11];
U(0.0, 0.0, 0.7853981633974483) qreg[11];
U(1.5707963267949, 0.0, 6.28318530717959) qreg[11];
cz qreg[9], qreg[11];
U(1.5707963267949, 3.14159265358979, 3.14159265358979) qreg[11];
U(0.0, 0.0, 3.14159265358979) qreg[9];
U(0.0, 0.0, 6.28318530717958) qreg[11];
U(0.0, 0.0, -0.39269908169872414) qreg[11];
U(1.570